In [2]:
%reset

# Questions before start:
We have a bunch of csv files, where each one is divided into categories:
1. Country (belgium, england, france, germany, greece, italy, netherlands, portugal, scotland, spain, turkey)
2. League (0, 1, 2, 3)
3. Season (2000-2001, 2001-2002, ..., 2021-2022)

Should we merge all the csv files into one big dataframe? Or should we keep them separate? 
If we merge them, how do we keep track of the different leagues and seasons?  

## Max's thoughts:
We could merge them into one big dataframe, and provide analysis on the whole dataset. 
In case we need analyse a specific league or season, we can filter the dataframe accordingly.

**Before merging**, we should add a column for the league and season.


In [7]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
raw_data = pd.read_csv("data_description/example.csv")

In [5]:
raw_data.head()

,Div,Date,HomeTeam,AwayTeam,Full time home goals,Full time away goals,Full time result,Half time home goals,Half time away goals,Half time result,...,Betbrain max >2.5,Betbrain average >2.5,Betbrain max <2.5,Betbrain average <2.5,Betbrain Asian bookies,Betbrain Asian home handicap,Betbrain max Asian home odds,Betbrain average Asian home odds,Betbrain max Asian away odds,Betbrain average Asian away odds
0,E0,13/08/05,Aston Villa,Bolton,2,2,D,2,2,D,...,2.20,2.01,1.87,1.70,22,-0.25,2.10,2.01,1.92,1.84
1,E0,13/08/05,Everton,Man United,0,2,A,0,1,A,...,2.10,1.93,1.87,1.79,23,0.75,2.05,2.00,1.93,1.86
2,E0,13/08/05,Fulham,Birmingham,0,0,D,0,0,D,...,2.24,2.04,1.77,1.69,21,0.00,1.85,1.81,2.11,2.05
3,E0,13/08/05,Man City,West Brom,0,0,D,0,0,D,...,2.10,1.94,1.90,1.77,23,-0.75,2.19,2.10,1.83,1.76
4,E0,13/08/05,Middlesboro,Liverpool,0,0,D,0,0,D,...,2.23,2.02,1.80,1.71,21,0.25,1.89,1.86,2.04,2.00
